## 동별 세대수 가져오기 [대구광역시]
+ 작성: 김지훈
+ 수정: 임경호

### 1. 지정한 년월에 사용 가능한 최신의 법정동, 행정동 코드 자료를 테이블에서 가져온다.

In [1]:
import pandas as pd
import requests
from tqdm.notebook import tqdm
import json
from datetime import datetime
import calendar

import sys
module_path = "D:\PythonProject\data-gatherer\common"
sys.path.append(module_path)
import dbconnect

In [2]:
region = "대구광역시"
#추출하고자 하는 년월
input_year = 2023
input_month = 8
# 년월: yyyymm 형식 문자열
month = str(input_year) + str(input_month).zfill(2)   
last_day = calendar.monthrange(input_year, input_month)[1]
# print(year, month, last_day)
month_end_date = datetime(input_year, input_month, last_day).strftime("%Y%m%d")

In [3]:
# 해당 년월에 사용가능한, 저장된 데이터의 마지막 일자 가져오기
conn = dbconnect.db_connect("DEMO_DM")
cur = conn.cursor()

query = "SELECT 업데이트일자 FROM city WHERE 업데이트일자 <= '" + month_end_date + "' and 시도명 = '" + region + "' ORDER BY 업데이트일자 DESC LIMIT 1"
cur.execute(query)
row = cur.fetchone()
stored_date = row[0]     

conn.close()

In [4]:
# 지역코드 테이블에서 데이터 읽어오기
engine = dbconnect.db_engine("DEMO_DM")
query = "SELECT * FROM city WHERE 업데이트일자 = '" + stored_date + "' and 시도명 = '" + region + "'"
result = pd.read_sql(query, engine)

In [5]:
df_code_table = result.copy()
df_code_table

,업데이트일자,행정동코드,시도명,시군구명,읍면동명,법정동코드,동리명
0,20230703,2700000000,대구광역시,,,2700000000,대구광역시
1,20230703,2711000000,대구광역시,중구,,2711000000,중구
2,20230703,2711051700,대구광역시,중구,동인동,2711010100,동인동1가
3,20230703,2711051700,대구광역시,중구,동인동,2711010200,동인동2가
4,20230703,2711051700,대구광역시,중구,동인동,2711010300,동인동3가
...,...,...,...,...,...,...,...
500,20230703,2772037000,대구광역시,군위군,삼국유사면,2772037027,양지리
501,20230703,2772037000,대구광역시,군위군,삼국유사면,2772037028,낙전리
502,20230703,2772037000,대구광역시,군위군,삼국유사면,2772037029,가암리
503,20230703,2772037000,대구광역시,군위군,삼국유사면,2772037030,석산리


### 2. 세대수 데이터 추출을 위한 법정동 코드 전처리

Open API를 사용하기 위해 **법정동 코드를 전처리**하는 과정입니다.  
- 해당 API는 법정동코드를 입력하였을 때, 값이 나오도록 되어 있습니다.  
- API사용 할 때, **구, 시, 리**의 법정동 코드를 제거하고 **동,읍,면** 법정동 코드만 남겨둬야합니다. 

법정동 코드는 총 10자리로 10자리의 구성은 아래와 같습니다.  
 - 시도(2) + 시군구(3) + 읍면동(3) + 리(2)

In [6]:
# 전제 코드 데이터
df_code_table_pre = df_code_table.copy()
df_code_table_pre.shape

(505, 7)

In [7]:
# '시군구' 단위 및 '읍면동' 단위의 데이터만 남겨놓는다
df_code_table_pre = df_code_table_pre[df_code_table_pre['법정동코드'].str.endswith('00')]
# '시군구' 단위 데이터 삭제
idx = df_code_table_pre[df_code_table_pre['법정동코드'].str.endswith('00000')].index
df_code_table_pre.drop(idx, inplace=True)
# 최종적으로 '읍면동' 코드만 남는다.
df_code_table_pre.shape

(304, 7)

### 각 지역별로 Data Set 구성

In [8]:
#구 별로 법정동코드를 추출하여 list형식으로 변환
def make_code_list_for_daegu(df,gu_name):
    df_gu= df[df['시군구명']==gu_name]
    list_gu=  df_gu['법정동코드']
    list_gu = list_gu.unique()
    return list_gu

In [9]:
# 각 도시별 법정동 코드 불러오기

#대구광역시
df_daegu_code = df_code_table_pre[df_code_table_pre['시도명'].str.contains('대구광역시')]
df_daegu_code= df_daegu_code.dropna(subset=['읍면동명']) #null값 제거
idx = df_daegu_code[df_daegu_code['법정동코드'].str[-4:] == "0000"].index #리 값 제거
df_daegu_code.drop(idx , inplace=True)

list_daegu_cd =  df_daegu_code['법정동코드']
list_daegu_cd = list_daegu_cd.unique() #대구광역시의 법정동코드

#중간에 오류가 날 수 있어 구별로 나누어 작업 진행
dict_region_codes = {}  # 딕셔너리
dict_region_codes["남구"] =  make_code_list_for_daegu(df_daegu_code,"남구")
dict_region_codes["달서구"] =  make_code_list_for_daegu(df_daegu_code,"달서구")
dict_region_codes["동구"] =  make_code_list_for_daegu(df_daegu_code,"동구")
dict_region_codes["북구"] =  make_code_list_for_daegu(df_daegu_code,"북구")
dict_region_codes["서구"] =  make_code_list_for_daegu(df_daegu_code,"서구")
dict_region_codes["수성구"] =  make_code_list_for_daegu(df_daegu_code,"수성구")
dict_region_codes["중구"] =  make_code_list_for_daegu(df_daegu_code,"중구")

In [10]:
# 각 지역별로 지역코드(동별) 개수 확인
for key in dict_region_codes:
    print(f'{key}\t{len(dict_region_codes[key])}')

남구	3
달서구	24
동구	45
북구	31
서구	9
수성구	26
중구	57


### 3. Open API를 이용한 세대수 데이터 추출

In [11]:
import warnings
from time import sleep
warnings.filterwarnings(action='ignore')

service_url = 'http://apis.data.go.kr/1741000/stdgPpltnHhStus'
# 일반 인증키(Encoding)	
api_key = 'IVgu%2FZBjA6hpLryyEOpySC2RhogOhaJIUqlXN8Uyj3Gxw4s3dX0qMxfgXMTLl60%2Fs2EYAMUsyyzTqwVOnjoIhg%3D%3D'
level = '4'     # 조회결과구분. 광역시도 단위 : 1, 시군구 단위 : 2, 읍면동 단위 : 3, 읍면동 통반 단위 : 4(기본값 : 4)
reg_code = '1'  # 등록구분. 전체:1, 거주자:2, 거주불명자:3, 재외국민:4(기본값 : 1)
req_type = 'json'   # 타입. XML, JSON(기본값 : XML)
pageRows = '8000'    # 페이지 크기. 페이지당 목록 수(1~100)(기본값 : 10)
pageNo = '1'        # 페이지 번호. 기본값 : 1

In [12]:
#법정동 코드를 이용한 행정동/법정동별 세대수를 API로 구하는 함수
def make_hh_data(df,list_cd,month):
    for i in tqdm(list_cd): 
        url = f'{service_url}/selectStdgPpltnHhStus?serviceKey={api_key}&stdgCd={i}&srchFrYm={month}&srchToYm={month}&lv={level}&regSeCd={reg_code}&type={req_type}&numOfRows={pageRows}&pageNo={pageNo}'
        # url = 'https://apis.data.go.kr/1741000/stdgPpltnHhStus/selectStdgPpltnHhStus?serviceKey=IVgu%2FZBjA6hpLryyEOpySC2RhogOhaJIUqlXN8Uyj3Gxw4s3dX0qMxfgXMTLl60%2Fs2EYAMUsyyzTqwVOnjoIhg%3D%3D&stdgCd='+i+'&srchFrYm='+month+'&srchToYm='+month+'&lv=4&regSeCd=1&type=json&numOfRows=8000&pageNo=1'
        response = requests.get(url,verify=False)
        #데이터 값 출력해보기
        contents = response.text
        json_ob = json.loads(contents)
        body = json_ob['Response']['items']['item']
        dataframe = pd.json_normalize(body)
        df = pd.concat([df,dataframe],axis = 0) #데이터 결합
    return df

#시군구별 세대수를 알아보는 함수 >> 시군구 자료와 데이터 비교
def know_hhcnt(df):
    df['hhCnt'] =df['hhCnt'].astype(str).astype(int)
    df['femlNmprCnt'] =df['femlNmprCnt'].astype(str).astype(int)
    df['totNmprCnt'] =df['totNmprCnt'].astype(str).astype(int)
    df['maleNmprCnt'] =df['maleNmprCnt'].astype(str).astype(int)
    know_hhcnt = df.groupby('sggNm').sum()
    return know_hhcnt

#기존의 표와 같이 보기 편하게 순서 변경 및 컬럼 변경
def name_sort_change(df):
    #이름 변경
    df.rename(columns={'hhCnt':'세대수','tong':'통','femlNmprCnt':'여자인구수','stdgCd':'법정동코드','maleFemlRate':'남여비율',
                       'stdgNm':'법정동명','ban':'반','totNmprCnt':'총인구수','ctpvNm':'시도명','maleNmprCnt':'남자인구수',
                       'sggNm':'시군구명','dongNm':'행정동명','hhNmpr':'세대당인구','admmCd':'행정기관코드',
                       'statsYm':'통계년월'},inplace=True)
    #기존의 표와 같이 순서 변경
    df = df[['통계년월','법정동코드','시도명','시군구명','법정동명','행정기관코드','행정동명','통',
             '반','총인구수','세대수','세대당인구','남자인구수','여자인구수','남여비율']]
    df['업데이트일자'] = datetime.now().strftime("%Y%m%d")
    return df

In [13]:
# 각 지역별로 데이터 수집 결과 초기화
dict_region_data = {}
dict_region_results = {}
for key in dict_region_codes:
    if key in ['중구']:
        dict_region_results[key] = 0
print(dict_region_results)

{'중구': 0}


In [15]:
# 각 지역별로 API 호출하여 동별 세대수 데이터 가져오기
for key in dict_region_results:
    if dict_region_results[key] == 0:
        # 최대 5회 시도
        num_retry = 10
        for retry in range(num_retry):
            sleep(5)
            try:
                print(key, " try ", retry)
                df_region_cd_list = dict_region_codes[key]
                df_region_data = pd.DataFrame()
                df_region_data = make_hh_data(df_region_data, df_region_cd_list, month) #남구
                dict_region_data[key] = df_region_data
                dict_region_results[key] = 1    # 성공
                break
            except Exception:
                continue            
    sleep(5)

중구  try  0


  0%|          | 0/57 [00:00<?, ?it/s]

중구  try  1


  0%|          | 0/57 [00:00<?, ?it/s]

중구  try  2


...

  0%|          | 0/57 [00:00<?, ?it/s]

중구  try  8


  0%|          | 0/57 [00:00<?, ?it/s]

중구  try  9


  0%|          | 0/57 [00:00<?, ?it/s]

In [29]:
# 각 지역별로 데이터 수집 결과 확인: 성공(1), 실패(0)
print("각 지역별 데이터 수집 결과")
for key in dict_region_results:
    print(f'{key}\t{dict_region_results[key]}')

각 지역별 데이터 수집 결과
중구	0


In [19]:
for key in dict_region_data:
    df = dict_region_data[key]
    print(key, "\t", df.shape[0])

동구 	 2599


In [20]:
df_house_hold = pd.DataFrame()
for key in dict_region_data:
    df_house_hold = pd.concat([df_house_hold, dict_region_data[key]], axis = 0)
df_house_hold.shape

(2599, 15)

In [21]:
df_house_hold

,hhCnt,tong,femlNmprCnt,stdgCd,maleFemlRate,stdgNm,ban,totNmprCnt,ctpvNm,maleNmprCnt,sggNm,dongNm,hhNmpr,admmCd,statsYm
0,20,1,18,2714010100,0.94,신암동,1,35,대구광역시,17,동구,신암2동,1.75,2714052000,202308
1,53,1,50,2714010100,0.90,신암동,1,95,대구광역시,45,동구,신암4동,1.79,2714054000,202308
2,51,1,41,2714010100,1.05,신암동,1,84,대구광역시,43,동구,신암3동,1.65,2714053000,202308
3,71,1,84,2714010100,0.95,신암동,1,164,대구광역시,80,동구,신암5동,2.31,2714054100,202308
4,55,1,29,2714010100,1.31,신암동,1,67,대구광역시,38,동구,신암1동,1.22,2714051000,202308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,87,45,95,2714014500,0.89,지묘동,3,180,대구광역시,85,동구,공산동,2.07,2714076000,202308
85,88,45,97,2714014500,0.91,지묘동,4,185,대구광역시,88,동구,공산동,2.10,2714076000,202308
86,104,45,116,2714014500,0.96,지묘동,5,227,대구광역시,111,동구,공산동,2.18,2714076000,202308
87,69,46,78,2714014500,0.95,지묘동,1,152,대구광역시,74,동구,공산동,2.20,2714076000,202308


In [32]:
# df_house_hold.drop_duplicates(keep = 'first',inplace=True) #중복값 제거 >> 중복값이 있을 때 첫번째 값만 남겨둠

In [22]:
print(know_hhcnt(df_house_hold))

        hhCnt                                               tong  femlNmprCnt  \
sggNm                                                                           
동구     161740  1111111111111111111111111111111122222222222222...       174250   

                                                  stdgCd  \
sggNm                                                      
동구     2714010100271401010027140101002714010100271401...   

                                            maleFemlRate  \
sggNm                                                      
동구     0.940.901.050.951.310.970.930.450.760.641.141....   

                                                  stdgNm  \
sggNm                                                      
동구     신암동신암동신암동신암동신암동신암동신암동신암동신암동신암동신암동신암동신암동신암동신암동신...   

                                                     ban  totNmprCnt  \
sggNm                                                                  
동구     1111122222333334444555556666677811111222223333...      341800 

In [23]:
#데이터 컬럼명 변경 및 위치 변경
df_house_hold = name_sort_change(df_house_hold)

In [24]:
df_house_hold.head()

,통계년월,법정동코드,시도명,시군구명,법정동명,행정기관코드,행정동명,통,반,총인구수,세대수,세대당인구,남자인구수,여자인구수,남여비율,업데이트일자
0,202308,2714010100,대구광역시,동구,신암동,2714052000,신암2동,1,1,35,20,1.75,17,18,0.94,20230908
1,202308,2714010100,대구광역시,동구,신암동,2714054000,신암4동,1,1,95,53,1.79,45,50,0.90,20230908
2,202308,2714010100,대구광역시,동구,신암동,2714053000,신암3동,1,1,84,51,1.65,43,41,1.05,20230908
3,202308,2714010100,대구광역시,동구,신암동,2714054100,신암5동,1,1,164,71,2.31,80,84,0.95,20230908
4,202308,2714010100,대구광역시,동구,신암동,2714051000,신암1동,1,1,67,55,1.22,38,29,1.31,20230908


# 5. 법정동, 행정동 세대수 데이터  SQL에 삽입 

In [25]:
conn = dbconnect.db_connect("DEMO_DW")
cursor = conn.cursor()

query = f'SELECT EXISTS (SELECT * FROM household_dong WHERE 통계년월 = {month})'

cursor.execute(query)
row = cursor.fetchone()
data_exist = row[0]     # 저장된 데이터의 유무(1 - 데이터 있음)

conn.close()

In [26]:
conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

for row in df_house_hold.itertuples():
    sql = "insert into household_dong (통계년월, 법정동코드, 시도명, 시군구명, 법정동명, \
                                        행정기관코드, 행정동명, 통, 반, 총인구수, 세대수, 세대당인구, \
                                        남자인구수, 여자인구수, 남여비율, 업데이트일자) \
            values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    cur.execute(sql, (row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9], row[10], row[11], row[12], row[13], row[14], row[15], row[16]))

conn.commit()
print(f'{month} 데이터를 저장하였습니다.')
conn.close()

202308 데이터를 저장하였습니다.
